# Create training dataset from varsom data


In [20]:
import pandas as pd
import numpy as np
import json
from flatten_json import flatten
from pprint import pprint
# make sure the aps module is in the pythonpath
import sys
from pathlib import Path

APS_dir = str(Path.cwd().parents[1])
if APS_dir not in sys.path:
    sys.path.append(APS_dir)
import aps.aps_io.get_forecasts as gf

In [21]:
# Nordvestlandet: Trollheimen, Romsdal, Sunnmøre
#warnings, url = gf.get_warnings_as_json([3022, 3023, 3024], "2017-12-01", "2018-05-31", lang_key=2, simple=False, recursive_count=5)

# Østlandet: Jotunheimen, Hallingdal, Vest-Telemark
warnings, url = gf.get_warnings_as_json([3028, 3032, 3035], "2017-12-01", "2018-05-31", lang_key=2, simple=False, recursive_count=5)


### Use this small data extraction for testing
#warnings, url = gf.get_warnings_as_json([3022], "2018-01-01", "2018-01-15", lang_key=2, simple=False, recursive_count=5)

print(url, '\n\n', type(warnings), len(warnings))

https://api01.nve.no/hydrology/forecast/avalanche/v4.0.0/api/AvalancheWarningByRegion/Detail/3035/2/2017-12-01/2018-05-31 

 <class 'list'> 546


In [22]:
# since get_warnings_as_json returns a list, we have to apply "flatten" to each item... 
warnings_flattened = (flatten(w, root_keys_to_ignore={'CountyList', 'MunicipalityList'}) for w in warnings)
# TODO: avalanche problems are not labeled correctly by their priority - need to correct.

In [23]:
df = pd.DataFrame(warnings_flattened)
df.head(5)

,Author,AvalancheDanger,AvalancheProblems_0_AvalCauseId,AvalancheProblems_0_AvalCauseName,AvalancheProblems_0_AvalProbabilityId,AvalancheProblems_0_AvalProbabilityName,AvalancheProblems_0_AvalPropagationId,AvalancheProblems_0_AvalPropagationName,AvalancheProblems_0_AvalTriggerSimpleId,AvalancheProblems_0_AvalTriggerSimpleName,...,RegionId,RegionName,RegionTypeId,RegionTypeName,SnowSurface,UtmEast,UtmNorth,UtmZone,ValidFrom,ValidTo
0,Espen Granan,Det ventes kaldt og rolig vintervær. I fjellet...,10,Buried weak layer of new snow,3,Possible,2,Some steep slopes,10,High additional load,...,3028,Jotunheimen,10,A,Total snødybde ca 50-100 cm. Det finnes løssnø...,155607,6844417,33,2017-12-01T00:00:00,2017-12-01T23:59:59
1,Espen Granan,Det ventes vestlig kuling utsatte steder i fje...,10,Buried weak layer of new snow,3,Possible,2,Some steep slopes,10,High additional load,...,3028,Jotunheimen,10,A,"Snødybde ca 50-150 cm, mest snø vest i regione...",155607,6844417,33,2017-12-02T00:00:00,2017-12-02T23:59:59
2,Espen Granan,Det ventes nysnø og kaldere temperatur. Dette ...,10,Buried weak layer of new snow,3,Possible,2,Some steep slopes,10,High additional load,...,3028,Jotunheimen,10,A,Lørdag passerte nedbør regionen med stigende t...,155607,6844417,33,2017-12-03T00:00:00,2017-12-03T23:59:59
3,Espen Granan,Det ventes nysnø og minusgrader. Dette kan gi ...,10,Buried weak layer of new snow,3,Possible,2,Some steep slopes,10,High additional load,...,3028,Jotunheimen,10,A,"Etter innslaget av mildere vær på lørdag, har ...",155607,6844417,33,2017-12-04T00:00:00,2017-12-04T23:59:59
4,Espen Granan,Mildvær og regn under ca 1100 moh vil gi økt s...,10,Buried weak layer of new snow,3,Possible,2,Some steep slopes,21,Low additional load,...,3028,Jotunheimen,10,A,Siste døgnet har det kommet litt nysnø vest i ...,155607,6844417,33,2017-12-05T00:00:00,2017-12-05T23:59:59


In [24]:
#df.columns.values

In [25]:
#df2 = df

In [26]:
#df = df2

In [27]:
# create necessary columns and populate them
for index, row in df.iterrows():
    for i in range(5):
        for j in range(4):
            try:
                col_name = '{MWType} {MWSubType}'.format(MWType=row['MountainWeather_MeasurementTypes_{0}_Name'.format(i)], MWSubType=row['MountainWeather_MeasurementTypes_{0}_MeasurementSubTypes_{1}_Name'.format(i, j)])
                col_name = col_name.replace(' ', '_')
                if col_name in df.columns.values:
                    df.loc[index, col_name] = row['MountainWeather_MeasurementTypes_{0}_MeasurementSubTypes_{1}_Value'.format(i, j)]
                else:
                    df[col_name] = np.nan
                    print('Created column: ', col_name)
                    df.loc[index, col_name] = row['MountainWeather_MeasurementTypes_{0}_MeasurementSubTypes_{1}_Value'.format(i, j)]
                    
                #print('{MWType} {MWSubType} = {MWSubTypeValue}'.format(MWType=df.loc[index]['MountainWeather_MeasurementTypes_{0}_Name'.format(i)],
                 #                                                      MWSubType=df.loc[index]['MountainWeather_MeasurementTypes_{0}_MeasurementSubTypes_{1}_Name'.format(i, j)],
                  #                                                     MWSubTypeValue=df.loc[index]['MountainWeather_MeasurementTypes_{0}_MeasurementSubTypes_{1}_Value'.format(i, j)]))
            except KeyError:
                print('MountainWeather_MeasurementTypes_{0}_MeasurementSubTypes_{1}... does not exist - moving on.'.format(i, j))

Created column:  Rainfall_Most_exposed_area
Created column:  Rainfall_Average
Created column:  Rainfall_nan
MountainWeather_MeasurementTypes_0_MeasurementSubTypes_3... does not exist - moving on.
Created column:  Wind_Speed
Created column:  Wind_Direction
Created column:  Wind_nan
MountainWeather_MeasurementTypes_1_MeasurementSubTypes_3... does not exist - moving on.
Created column:  Wind_shift_to_Speed
Created column:  Wind_shift_to_Direction
Created column:  Wind_shift_to_Timeperiode_start
Created column:  Wind_shift_to_Timeperiode_end
Created column:  Temperature_Min
Created column:  Temperature_Max
Created column:  Temperature_masl
MountainWeather_MeasurementTypes_3_MeasurementSubTypes_3... does not exist - moving on.
Created column:  Freezing_Level_masl
Created column:  Freezing_Level_Timeperiode_start
Created column:  Freezing_Level_Timeperiode_end
Created column:  Freezing_Level_nan
MountainWeather_MeasurementTypes_0_MeasurementSubTypes_3... does not exist - moving on.
MountainW

In [28]:
df.columns.values

array(['Author', 'AvalancheDanger', 'AvalancheProblems_0_AvalCauseId',
       'AvalancheProblems_0_AvalCauseName',
       'AvalancheProblems_0_AvalProbabilityId',
       'AvalancheProblems_0_AvalProbabilityName',
       'AvalancheProblems_0_AvalPropagationId',
       'AvalancheProblems_0_AvalPropagationName',
       'AvalancheProblems_0_AvalTriggerSimpleId',
       'AvalancheProblems_0_AvalTriggerSimpleName',
       'AvalancheProblems_0_AvalancheAdvice',
       'AvalancheProblems_0_AvalancheExtId',
       'AvalancheProblems_0_AvalancheExtName',
       'AvalancheProblems_0_AvalancheProblemId',
       'AvalancheProblems_0_AvalancheProblemTypeId',
       'AvalancheProblems_0_AvalancheProblemTypeName',
       'AvalancheProblems_0_AvalancheTypeId',
       'AvalancheProblems_0_AvalancheTypeName',
       'AvalancheProblems_0_DestructiveSizeExtId',
       'AvalancheProblems_0_DestructiveSizeExtName',
       'AvalancheProblems_0_ExposedHeight1',
       'AvalancheProblems_0_ExposedHeight2',
    

In [29]:
df[['ValidFrom',
    'Temperature_Max',
    'Temperature_Min',
    'Wind_Direction',
    'Wind_Speed']].head(12)

,ValidFrom,Temperature_Max,Temperature_Min,Wind_Direction,Wind_Speed
0,2017-12-01T00:00:00,-5,-17,SW,Breeze
1,2017-12-02T00:00:00,-1,-7,W,Moderate gale
2,2017-12-03T00:00:00,-5,-10,W,Strong breeze
3,2017-12-04T00:00:00,-5,-11,W,Strong breeze
4,2017-12-05T00:00:00,-2,-8,W,Moderate gale
5,2017-12-06T00:00:00,-1,-9,SW,Breeze
6,2017-12-07T00:00:00,1,-8,S,Moderate gale
7,2017-12-08T00:00:00,-5,-9,W,Fresh breeze
8,2017-12-09T00:00:00,-6,-10,W,Strong breeze
9,2017-12-10T00:00:00,.9,-16,SW,Fresh breeze


In [30]:
# store all data before filtering and splitting
df.to_csv('varsom_all.csv', index_label='index')

# Create training data
We want to predict the main avalanche problem. Therefore we store the AP1_type and AP1_score in its own dataframe.
We remove all parameters that are irrelevant for the algorithm, e.g. UtmZone, ValidTo...

### Make all columns used in the analysis numerical and adjust indicies where necessary.

In [31]:
with open(r'../config/snoskred_keys.json') as jdata:
    snoskred_keys = json.load(jdata)

pprint(snoskred_keys)

{'': {},
 'AvalPropagationId': {'0': 'Not given',
                       '1': 'Isolated steep slopes',
                       '2': 'Some steep slopes',
                       '3': 'Many steep slopes',
                       '4': 'Most steep slopes',
                       '5': 'Also in moderately steep terrain'},
 'AvalTriggerSimpleId': {'0': 'Not given',
                         '10': 'High additional load',
                         '21': 'Low additional load',
                         '22': 'Spontaneous release'},
 'AvalancheProblemTypeId': {'0': 'Ikke gitt',
                            '10': 'Wind slab avalanches',
                            '20': 'New snow',
                            '3': 'Loose dry avalanches',
                            '30': 'Persistent slab avalanches',
                            '37': 'Persistent deep slab avalanches',
                            '40': 'Wet snow',
                            '45': 'Wet slab avalanches',
                            '5': 'L

In [32]:
print(df['AvalancheProblems_1_AvalPropagationId'].unique(), type(df['AvalancheProblems_1_AvalPropagationId'].unique()[0]))


[nan  2.  1.  3.] <class 'numpy.float64'>


In [33]:
# change wind speeds to numerical values
df['Wind_Speed_Num'] = df['Wind_Speed'].apply(lambda i: snoskred_keys['beaufort_scale_en'][i])
df['Wind_Direction_Num'] = df['Wind_Direction'].apply(lambda i: 0 if i == None else snoskred_keys['wind_dir_conv_en'][i])

# Re-group AvalancheProblemType
# AvalancheProblemType grouped by PWL, wet slab, wet loose, dry loose, storm slab, and wind slab (and glide avalanche).
df['AvalancheProblems_0_Class_AvalancheProblemTypeId'] = df['AvalancheProblems_0_AvalancheProblemTypeId'].apply(lambda i: 0 if i == np.nan else np.int(snoskred_keys['Class_AvalancheProblemTypeId'][str(int(i))]))
df['AvalancheProblems_1_Class_AvalancheProblemTypeId'] = df['AvalancheProblems_1_AvalancheProblemTypeId'].apply(lambda i: 0 if str(i) == str(np.nan) else np.int(snoskred_keys['Class_AvalancheProblemTypeId'][str(int(i))]))

# Distribution is labeled _Propagation_ in the API and has five classes. Change name to _AvalDistribution_ and merge the uper three classes into one called _widespread_.
df['AvalancheProblems_0_Class_AvalDistributionId'] = df['AvalancheProblems_0_AvalPropagationId'].apply(lambda i: 0 if str(i) == str(np.nan) else np.int(snoskred_keys['Class_AvalDistributionId'][str(int(i))]))
df['AvalancheProblems_1_Class_AvalDistributionId'] = df['AvalancheProblems_1_AvalPropagationId'].apply(lambda i: 0 if str(i) == str(np.nan) else np.int(snoskred_keys['Class_AvalDistributionId'][str(int(i))]))

In [34]:
# Keep only columns that hold numerical values.
# AvalCause, AvalancheExt and AvalancheProblemType are directly correlated - keep only re-grouped ..._Class_AvalancheProblemType.
df_numdata = df.filter(['AvalancheProblems_0_AvalProbabilityId',
                        'AvalancheProblems_0_Class_AvalDistributionId',
                        'AvalancheProblems_0_AvalTriggerSimpleId',
                        'AvalancheProblems_0_AvalancheProblemId',
                        'AvalancheProblems_0_Class_AvalancheProblemTypeId',
                        'AvalancheProblems_0_AvalancheTypeId',
                        'AvalancheProblems_0_DestructiveSizeExtId',
                        'AvalancheProblems_1_AvalProbabilityId',
                        'AvalancheProblems_1_Class_AvalDistributionId',
                        'AvalancheProblems_1_AvalTriggerSimpleId',
                        'AvalancheProblems_1_AvalancheProblemId',
                        'AvalancheProblems_1_Class_AvalancheProblemTypeId',
                        'AvalancheProblems_1_AvalancheTypeId',
                        'AvalancheProblems_1_DestructiveSizeExtId',
                        'DangerLevel',
                        'ValidFrom',
                        'Rainfall_Most_exposed_area',
                        'Rainfall_Average',
                        'Wind_Speed_Num',
                        'Wind_Direction_Num',
                        'Temperature_Min',
                        'Temperature_Max',
                        'Temperature_masl',
                        'Freezing_Level_masl'], axis=1)

In [38]:
df_numdata.fillna(0, inplace=True)

In [39]:
# Check if there are no weired values.
for col in df_numdata.drop(['ValidFrom'], axis=1).columns.values:
    print(col, ': ', df_numdata[col].unique())

AvalancheProblems_0_AvalProbabilityId :  [3 2 5]
AvalancheProblems_0_Class_AvalDistributionId :  [2 3 1]
AvalancheProblems_0_AvalTriggerSimpleId :  [10 21 22]
AvalancheProblems_0_AvalancheProblemId :  [1 2 3]
AvalancheProblems_0_Class_AvalancheProblemTypeId :  [6 5 7 4 2]
AvalancheProblems_0_AvalancheTypeId :  [10 20]
AvalancheProblems_0_DestructiveSizeExtId :  [2 1 3]
AvalancheProblems_1_AvalProbabilityId :  [0. 3. 2. 5.]
AvalancheProblems_1_Class_AvalDistributionId :  [0 2 1 3]
AvalancheProblems_1_AvalTriggerSimpleId :  [ 0. 10. 21. 22.]
AvalancheProblems_1_AvalancheProblemId :  [0. 2. 1. 3.]
AvalancheProblems_1_Class_AvalancheProblemTypeId :  [0 7 4 6 5 2]
AvalancheProblems_1_AvalancheTypeId :  [ 0. 10. 20.]
AvalancheProblems_1_DestructiveSizeExtId :  [0. 2. 1. 3.]
DangerLevel :  ['2' '3' '1']
Rainfall_Most_exposed_area :  ['0' '6' '15' '7' '16' '25' '20' '30' '2' '4' '3' '45' '10' '5' '12' 0
 '14' '1' '8' '18' '50' '9' '35' '40' '13']
Rainfall_Average :  ['0' '2' '1' '5' '8' '10' '

In [40]:
# Remove all columns without data
#df_numdata = df_numdata.dropna(axis=1)

df_numdata.to_csv('varsom_numdata.csv', index_label='index')

## Split data into a training- and test-dataset
Randomly choose indicies that should serve a test data and which are removed from the training data.
Maybe write a function that chooses a certain % as test dxata

In [16]:
# Randomly shuffle the index of nba.
random_indices = np.random.permutation(df_numdata.index)
# Set a cutoff for how many items we want in the test set (in this case 1/3 of the items)
test_cutoff = np.int(np.floor(len(df_numdata)/3))
print(test_cutoff)
# Generate the test set by taking the first 1/3 of the randomly shuffled indices.
df_test = df_numdata.loc[random_indices[1:test_cutoff]]
# Generate the train set with the rest of the data.
df_train = df_numdata.loc[random_indices[test_cutoff:]]

182


Separate the target variable in its own dataframe.

In [17]:
df_train_data = df_train.drop(['DangerLevel'], axis=1)
df_test_data = df_test.drop(['DangerLevel'], axis=1)

df_train_target = df_train.filter(['DangerLevel'], axis=1)
df_test_target = df_test.filter(['DangerLevel'], axis=1)

In [18]:
df_train_data.to_csv('varsom_train_data.csv', index_label='index')
df_test_data.to_csv('varsom_test_data.csv', index_label='index')

df_train_target.to_csv('varsom_train_target.csv', index_label='index')
df_test_target.to_csv('varsom_test_target.csv', index_label='index')